# Imports and package installation

In [1]:
# imports 
from pydub import AudioSegment
from google.cloud import speech
from huggingsound import SpeechRecognitionModel
import os
import io
import pandas as pd


/home/alexandre/.pyenv/versions/3.8.12/envs/hear2me/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
#installs
#!pip install pydub
#!pip install leia
#!pip install nltk
#!pip install hurggingsound
#!pip install vaderSentiment
#!pip install transformers
#!pip install spacy

# Data Import and processing

In [3]:
#import tsv with data
text_df = pd.read_csv('../raw_data/commonvoicedataset/validated.tsv', sep='\t', header=0, low_memory=False)

In [4]:
#check df
text_df.head(2)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment
0,09e8441e62e3c8da70b667874fa75e96731f6e43e359a1...,common_voice_pt_27283586.mp3,"Se esta primeira condição for satisfeita, é se...",2,0,NaN,NaN,NaN,pt,NaN
1,12baee49ea5072cfd8392765aeb6d0e518a51a23224aa8...,common_voice_pt_25643625.mp3,Nós temos tempo suficiente.,2,1,NaN,NaN,NaN,pt,NaN


In [5]:
# create dict with samples file name and sentence
sample = {}
for p, s in text_df[['path','sentence']].iloc[0:100].itertuples(index=False):
    sample[p] = s

In [6]:
#convert files to wav
for file in sample:
    sound = AudioSegment.from_mp3(f"../raw_data/commonvoicedataset/clips/{file}")
    sound = sound.set_frame_rate(16000)
    sound.export(f"../raw_data/{file[:-4]}.wav", format="wav")

In [7]:
# save audios names
audio_names = list(sample.keys())

In [8]:
#create paths to audio
audio_paths = []
for name in audio_names:
    audio_paths.append(f"../raw_data/{name[:-4]}.wav")

# Wav2vec Models for transcribing audio

In [9]:
#use Wav2Vec model from huggingsound
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-portuguese")

transcriptions = model.transcribe(audio_paths)

04/07/2022 20:46:55 - INFO - huggingsound.speech_recognition.model - Loading model...


100%|████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.33s/it]


In [10]:
#create list of transcriptions results
transcriptions_list = []
for transcript in transcriptions:
    transcriptions_list.append(transcript['transcription'])

In [11]:
#create sample dataframe with filename and sentence
sample_df = pd.DataFrame(list(sample.items()), columns=['file_name', 'sentence'])

In [12]:
#add transcribed sentence list
sample_df['transc_sentence'] = transcriptions_list

In [13]:
#check sample df
sample_df.head()

,file_name,sentence,transc_sentence
0,common_voice_pt_27283586.mp3,"Se esta primeira condição for satisfeita, é se...",se esta família condição for satisfeita é sens...
1,common_voice_pt_25643625.mp3,Nós temos tempo suficiente.,nós temos tempos sob istelan
2,common_voice_pt_27023139.mp3,"Bem, digitalizar Don Quixote é um passo para l...",bem ditalizar do ke shot é un passo para levar...
3,common_voice_pt_30386095.mp3,Ariranha,aria venho
4,common_voice_pt_24001131.mp3,Ele é advogado do comando vermelho.,ele é advogado do comando vermelho


In [14]:
#check true sentences vs transcribed sentences 
for i, j in sample_df.iterrows():
    print(i, j)

0 file_name                               common_voice_pt_27283586.mp3
sentence           Se esta primeira condição for satisfeita, é se...
transc_sentence    se esta família condição for satisfeita é sens...
Name: 0, dtype: object
1 file_name          common_voice_pt_25643625.mp3
sentence            Nós temos tempo suficiente.
transc_sentence    nós temos tempos sob istelan
Name: 1, dtype: object
2 file_name                               common_voice_pt_27023139.mp3
sentence           Bem, digitalizar Don Quixote é um passo para l...
transc_sentence    bem ditalizar do ke shot é un passo para levar...
Name: 2, dtype: object
3 file_name          common_voice_pt_30386095.mp3
sentence                               Ariranha
transc_sentence                      aria venho
Name: 3, dtype: object
4 file_name                 common_voice_pt_24001131.mp3
sentence           Ele é advogado do comando vermelho.
transc_sentence     ele é advogado do comando vermelho
Name: 4, dtype: object
5 file_n

In [16]:
#test another model

from transformers import AutoTokenizer, Wav2Vec2ForCTC
  
tokenizer = AutoTokenizer.from_pretrained("Edresson/wav2vec2-large-xlsr-coraa-portuguese")

model_2 = SpeechRecognitionModel("Edresson/wav2vec2-large-xlsr-coraa-portuguese")

/home/alexandre/.pyenv/versions/3.8.12/envs/hear2me/lib/python3.8/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


04/07/2022 20:49:28 - INFO - huggingsound.speech_recognition.model - Loading model...


In [17]:
transcriptions_2 = model_2.transcribe(audio_paths)

100%|████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.31s/it]


## Comparing transcriptions to actual sentences

In [18]:
#create list of transcriptions results
transcriptions_list_2 = []
for transcript in transcriptions_2:
    transcriptions_list_2.append(transcript['transcription'])
    
#create sample dataframe with filename and sentence
sample_df_2 = pd.DataFrame(list(sample.items()), columns=['file_name', 'sentence'])

#add transcribed sentence list
sample_df_2['transc_sentence'] = transcriptions_list_2

#check true sentences vs transcribed sentences 
for i, j in sample_df_2.iterrows():
    print(i, j)

0 file_name                               common_voice_pt_27283586.mp3
sentence           Se esta primeira condição for satisfeita, é se...
transc_sentence    se esta primília condição for satisfeita é sen...
Name: 0, dtype: object
1 file_name          common_voice_pt_25643625.mp3
sentence            Nós temos tempo suficiente.
transc_sentence        noss tes tempos estunone
Name: 1, dtype: object
2 file_name                               common_voice_pt_27023139.mp3
sentence           Bem, digitalizar Don Quixote é um passo para l...
transc_sentence    bem diritalizar dom que shat é um passo para l...
Name: 2, dtype: object
3 file_name          common_voice_pt_30386095.mp3
sentence                               Ariranha
transc_sentence                     aris aí não
Name: 3, dtype: object
4 file_name                 common_voice_pt_24001131.mp3
sentence           Ele é advogado do comando vermelho.
transc_sentence     ele é advogado do comando vermelho
Name: 4, dtype: object
5 file_n

In [19]:
#import audio from Senna speech
senna_sound = AudioSegment.from_ogg("../raw_data/senna.ogg")
senna_sound = senna_sound.set_frame_rate(16000)
senna_sound.export(f"../raw_data/senna.wav", format="wav")

<_io.BufferedRandom name='../raw_data/senna.wav'>

In [20]:
# Transcribe Senna audio with model 1
senna_transc = model.transcribe(["../raw_data/senna.wav"])
senna_transc[0]['transcription']

100%|████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.43s/it]


'você tem que provar primeiro para consegui o apoio e você psãdo a coipa chegar lá abrigação que eu tem e barl máximo de wim e al ento a luto procurasse campeãor eu acredito que rretom canções de lutário gualpelhor eu tenho canções de lutar com com qualquer outo piloto e areditou nisso é que eu procuro tirar o máximo de mim isso começo a jano nora se faz preparando sicologicamente e essa vontade de vencei ao que eu mantém e a minha ultimação maiora vontade me enciono quem mantém participando num no ima corrida nunca pat palmãl'

In [21]:
# Transcribe Senna audio with model 2
senna_transc_2 = model_2.transcribe(["../raw_data/senna.wav"])
senna_transc_2[0]['transcription']

100%|████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.58s/it]


'você tem que provar primeiro para conseguir o apoio e vocêcê precisa o apoio pra chegar lá a obrigação que eu tenho ele dá o máximo de mim e realmente a luta procurarxia quio campo eu acredito que que a gente tem condições de lutar igual pra egor que eu tenho condições de lutar com com qualquer outro piloto e acreditando isso é que eu procuro tirar o máximo de mim isso começa já na hora que você taz preparando psicologicamente e essa vontade de vencer o que mantem e a minha moltivação maior a vontade de me encima quem mantem participando nu no numa corrida u nunca nnca la de formar um'

# LDA Approach to summarization

## Text pre-processing

In [23]:
#clean text

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

#remove stop words

stop_words = stopwords.words('portuguese')

def rmv_sw(text):
    word_tokens = word_tokenize(text) 
    text = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(text)

texto = rmv_sw(senna_transc_2[0]['transcription'])
texto

'provar primeiro conseguir apoio vocêcê precisa apoio pra chegar lá obrigação dá máximo mim realmente luta procurarxia quio campo acredito gente condições lutar igual pra egor condições lutar qualquer outro piloto acreditando procuro tirar máximo mim começa hora taz preparando psicologicamente vontade vencer mantem moltivação maior vontade encima mantem participando nu corrida u nunca nnca la formar'

## LDA Model

In [25]:
#usar LDA para extrair principais palavras do texto

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df = 2).fit([texto])

data_vectorized = vectorizer.transform([texto])

lda_model = LatentDirichletAllocation(n_components=1).fit(data_vectorized)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('vontade', 1.2357022603955121), ('condições', 1.2357022603955121), ('máximo', 1.2357022603955121), ('lutar', 1.2357022603955121), ('pra', 1.2357022603955121), ('mim', 1.2357022603955121), ('mantem', 1.2357022603955121), ('apoio', 1.2357022603955121), ('começa', 1.1178511301977558), ('conseguir', 1.1178511301977558)]


/home/alexandre/.pyenv/versions/3.8.12/envs/hear2me/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
#fazer lista com principais palavras
main_topics = []
for idx, topic in enumerate(lda_model.components_):
    main_topics.append([(vectorizer.get_feature_names_out()[i])
                    for i in topic.argsort()[:-10 - 1:-1]])

In [27]:
#printar principais palavras
print(main_topics[0])

['vontade', 'condições', 'máximo', 'lutar', 'pra', 'mim', 'mantem', 'apoio', 'começa', 'conseguir']


# Sentiment analysis

In [54]:
#Sentiment analysis

#Use portuguese sentiment analysis lib

from libs.leia.leia import SentimentIntensityAnalyzer

s = SentimentIntensityAnalyzer()

#test sentiment analysis for each audio

lista = list(sample_df['sentence'])

for frase in lista:
    print(frase, s.polarity_scores(frase))

Se esta primeira condição for satisfeita, é sensata, forte e ágil. {'neg': 0.158, 'neu': 0.842, 'pos': 0.0, 'compound': -0.128}
Nós temos tempo suficiente. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Bem, digitalizar Don Quixote é um passo para levar a cultura a todos. {'neg': 0.137, 'neu': 0.684, 'pos': 0.179, 'compound': 0.128}
Ariranha {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Ele é advogado do comando vermelho. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
São José do Brejo do Cruz {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Oi, meu nome é Betina {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Nem o radar nem os outros instrumentos detectaram o bombardeiro stealth. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Pedir dinheiro emprestado às pessoas da aldeia {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
oito {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Trancá-los {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.

# Spacy library approach to text pre processing

In [63]:
#use this in bash to download portuguese pipeline: python -m spacy download pt_core_news_lg

In [64]:
#import spacy and load portuguese model

import spacy

nlp = spacy.load("pt_core_news_lg")

In [92]:
#save text as spacy object
texto_completo = nlp(senna_transc_2[0]['transcription'])

In [76]:
#extract tokens from text

for token in texto_completo:
    print (token, token.idx)

você 0
tem 5
que 9
provar 13
primeiro 20
para 29
conseguir 34
o 44
apoio 46
e 52
vocêcê 54
precisa 61
o 69
apoio 71
pra 77
chegar 81
lá 88
a 91
obrigação 93
que 103
eu 107
tenho 110
ele 116
dá 120
o 123
máximo 125
de 132
mim 135
e 139
realmente 141
a 151
luta 153
procurarxia 158
quio 170
campo 175
eu 181
acredito 184
que 193
que 197
a 201
gente 203
tem 209
condições 213
de 223
lutar 226
igual 232
pra 238
egor 242
que 247
eu 251
tenho 254
condições 260
de 270
lutar 273
com 279
com 283
qualquer 287
outro 296
piloto 302
e 309
acreditando 311
isso 323
é 328
que 330
eu 334
procuro 337
tirar 345
o 351
máximo 353
de 360
mim 363
isso 367
começa 372
já 379
na 382
hora 385
que 390
você 394
taz 399
preparando 403
psicologicamente 414
e 431
essa 433
vontade 438
de 446
vencer 449
o 456
que 458
mantem 462
e 469
a 471
minha 473
moltivação 479
maior 490
a 496
vontade 498
de 506
me 509
encima 512
quem 519
mantem 524
participando 531
nu 544
no 547
numa 550
corrida 555
u 563
nunca 565
nnca 571
la 576
de 

In [70]:
#save portuguese stopwords

spacy_stopwords = spacy.lang.pt.stop_words.STOP_WORDS

In [77]:
#remove stopwords from text

for token in texto_completo:
    if not token.is_stop:
        print (token)

provar
conseguir
vocêcê
precisa
pra
chegar
obrigação
mim
realmente
luta
procurarxia
quio
campo
acredito
gente
condições
lutar
igual
pra
egor
condições
lutar
outro
piloto
acreditando
procuro
tirar
mim
começa
hora
taz
preparando
psicologicamente
vontade
vencer
mantem
moltivação
vontade
encima
mantem
participando
nu
corrida
u
nnca
la
formar


In [95]:
#lemmatize words

for token in texto_completo:
    print (token, token.lemma_)

você você
tem ter
que que
provar provar
primeiro primeiro
para parir
conseguir conseguir
o o
apoio apoiar
e e
vocêcê vocêcê
precisa preciso
o o
apoio apoiar
pra pra
chegar chegar
lá lá
a o
obrigação obrigação
que que
eu eu
tenho ter
ele ele
dá dar
o o
máximo máximo
de de
mim mim
e e
realmente realmente
a o
luta lutar
procurarxia procurarxia
quio quio
campo campar
eu eu
acredito acreditar
que que
que que
a o
gente gente
tem ter
condições condição
de de
lutar lutar
igual igual
pra pra
egor egor
que que
eu eu
tenho ter
condições condição
de de
lutar lutar
com com
com com
qualquer qualquer
outro outro
piloto pilotar
e e
acreditando acreditar
isso isso
é ser
que que
eu eu
procuro procurar
tirar tirar
o o
máximo máximo
de de
mim mim
isso isso
começa comedir
já já
na o
hora horar
que que
você você
taz taz
preparando preparar
psicologicamente psicologicamente
e e
essa esse
vontade vontade
de de
vencer vencer
o o
que que
mantem mantar
e e
a o
minha meu
moltivação moltivação
maior maior
a o
vont

In [81]:
# Part of Speech Tagging

for token in texto_completo:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

você PRON PRON pronoun
tem AUX AUX auxiliary
que SCONJ SCONJ subordinating conjunction
provar VERB VERB verb
primeiro ADV ADV adverb
para SCONJ SCONJ subordinating conjunction
conseguir VERB VERB verb
o DET DET determiner
apoio NOUN NOUN noun
e CCONJ CCONJ coordinating conjunction
vocêcê PROPN PROPN proper noun
precisa VERB VERB verb
o DET DET determiner
apoio NOUN NOUN noun
pra SCONJ SCONJ subordinating conjunction
chegar VERB VERB verb
lá ADV ADV adverb
a DET DET determiner
obrigação NOUN NOUN noun
que PRON PRON pronoun
eu PRON PRON pronoun
tenho VERB VERB verb
ele PRON PRON pronoun
dá VERB VERB verb
o DET DET determiner
máximo NOUN NOUN noun
de ADP ADP adposition
mim PRON PRON pronoun
e CCONJ CCONJ coordinating conjunction
realmente ADV ADV adverb
a DET DET determiner
luta NOUN NOUN noun
procurarxia ADJ ADJ adjective
quio PROPN PROPN proper noun
campo PROPN PROPN proper noun
eu PRON PRON pronoun
acredito VERB VERB verb
que SCONJ SCONJ subordinating conjunction
que SCONJ SCONJ subord

In [203]:
#function to extract full name from text

from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

def extract_full_name(nlp_doc):
    pattern = [[{'POS': 'PROPN'}, {'POS': 'PROPN'}]]
    matcher.add('FULL_NAME', pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text

In [204]:
texto_test = nlp('''Testando capacidade do spacy de detectar nome dentro 
                 de um texto, Alexandre Carvalho testando pra ver se funciona''')

extract_full_name(texto_test)

'Alexandre Carvalho'

In [93]:
# displacy Visualization

from spacy import displacy

#displacy.serve(texto_completo, style='dep')

In [158]:
from heapq import nlargest
from spacy.lang.pt.stop_words import STOP_WORDS

#function for counting word frequency

def word_freq(text, count):
    nlp = spacy.load('pt_core_news_lg')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    return nlargest(count, word_frequencies)

In [162]:
nlp = spacy.load('pt_core_news_lg')
doc= nlp(texto_completo)
tokens=[token.text for token in doc]
word_frequencies={}
for word in doc:
    if word.text.lower() not in list(STOP_WORDS):
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
max_frequency=max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency

In [164]:
word_freq(texto_completo, 3)

['vontade', 'vocêcê', 'vencer']

In [163]:
word_frequencies

{'provar': 0.5,
 'conseguir': 0.5,
 'vocêcê': 0.5,
 'precisa': 0.5,
 'pra': 1.0,
 'chegar': 0.5,
 'obrigação': 0.5,
 'mim': 1.0,
 'realmente': 0.5,
 'luta': 0.5,
 'procurarxia': 0.5,
 'quio': 0.5,
 'campo': 0.5,
 'acredito': 0.5,
 'gente': 0.5,
 'condições': 1.0,
 'lutar': 1.0,
 'igual': 0.5,
 'egor': 0.5,
 'outro': 0.5,
 'piloto': 0.5,
 'acreditando': 0.5,
 'procuro': 0.5,
 'tirar': 0.5,
 'começa': 0.5,
 'hora': 0.5,
 'taz': 0.5,
 'preparando': 0.5,
 'psicologicamente': 0.5,
 'vontade': 1.0,
 'vencer': 0.5,
 'mantem': 1.0,
 'moltivação': 0.5,
 'encima': 0.5,
 'participando': 0.5,
 'nu': 0.5,
 'corrida': 0.5,
 'u': 0.5,
 'nnca': 0.5,
 'la': 0.5,
 'formar': 0.5}

# To Do

In [96]:
# Treinar word2vec na lista de palavras do stopwords pra poder aproximar palavras que não estão presentes na lista